<a href="https://www.kaggle.com/code/sachinfulare/sachin-bankmarketing?scriptVersionId=162968292" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Goal :- Find the best strategies to improve for the next marketing campaign.
Find the best strategies to improve for the next marketing campaign. How can the financial institution have a greater effectiveness for future marketing campaigns? 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import seaborn as sns
from scipy import stats

# 1) loading data set

In [ ]:
data = pd.read_csv("/kaggle/input/bank-marketing-dataset/bank.csv")
print("successfully loaded")

# 2) Getting general overview about the data

In [ ]:
data

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.columns

In [ ]:
data.shape

# 3) Exploratory data analysis
1.Analysis of data

2.Handling missing values

3.Visualisation

In [ ]:
data.describe(include='all').T

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

# Visualisation

In [ ]:
data.deposit.hist()
plot.title("deposit-frequency graph")
plot.xlabel("deposit")
plot.ylabel("frequency")

In [ ]:
categorical_cols=list()
numerical_cols=list()
for i in data.columns :
    if data[i].dtype == "object" :
        categorical_cols.append(i)
    if data[i].dtype != "object" :
        numerical_cols.append(i)
print(categorical_cols)
print(numerical_cols)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plot

for i in data.columns :
    if data[i].dtype != "object":
        print(i)
        sns.scatterplot(data=data,x=i,y="deposit",hue="deposit")
        plot.title("Scatter plot of deposit and "+str(i))
        plot.xlabel(i)
        plot.ylabel("deposit")
        plot.show()

Insight: Looking into scatter plots/graphs, i can conlcude that numerical coloumsn has various range of values

In [ ]:
for i in data.columns :
    if data[i].dtype != "object":
        print(i)
        sns.boxplot(data=data,x=i)
        plot.title("box plot")
        plot.show()

# Handling Outlier by IQR Method

In [ ]:
# Define a function to remove outliers using the IQR method
def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# Create boxplots before and after removing outliers
for col in data.columns:
    if data[col].dtype != "object":
        plot.figure(figsize=(12, 4))

        # Boxplot before removing outliers
        plot.subplot(1, 2, 1)
        data.boxplot(column=col)
        plot.title(f'{col} Boxplot (Before Removing Outliers)')

        # Remove outliers
        outlr_removed = remove_outliers(data, col)

        # Boxplot after removing outliers
        plot.subplot(1, 2, 2)
        outlr_removed.boxplot(column=col)
        plot.title(f'{col} Boxplot (After Removing Outliers)')

plot.show()

In [ ]:
outlr_removed

In [ ]:
outlr_removed.shape

# Preprocessing

# ** Data Encoding **

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_outlook = LabelEncoder()
encodedData=outlr_removed.copy()
for i in encodedData.columns :
    if encodedData[i].dtype == "object" :
        S=encodedData[i].unique()
        S.sort()
        print(S)

        encodedData[i]=le_outlook.fit_transform(encodedData[i])

        encodedData[i].hist()
        plot.xlabel(i)
        plot.ylabel("frequency")
        plot.show()

In [ ]:
categorical_cols

In [ ]:
numerical_cols

In [ ]:
encodedData

I dropped 'poutcome' column due to it includes more frequncy of 'unknown' values.

In [ ]:
encodedData = encodedData.drop(columns='poutcome',axis=1)
encodedData

In [ ]:
plot.figure(figsize=(15,10))
heat_map=sns.heatmap(encodedData.corr(),annot=True)
plot.show()

Insight: 'pdays' and 'previous' variable has more correlation 

loan has less correlation with 'deposit'

# **Feature Scaling**

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
def scaling_fun(data,col):
    min_max_scaler=MinMaxScaler()
    min_max_scaler.fit(data[col])
    return min_max_scaler

In [ ]:
# processing the data into MinMaxScaler modle
scaler_modle=scaling_fun(encodedData.copy(),numerical_cols)
scaled_data=pd.DataFrame(scaler_modle.transform(encodedData[numerical_cols]))
scaled_data.columns=encodedData[numerical_cols].columns #since we scaled only the numerical values 
                                                        #we need to concatenate to actual dataframe
scaled_data

In [ ]:
cmpltScaledData=encodedData.copy()
cmpltScaledData

In [ ]:
cmpltScaledData.index=scaled_data.index  #replace the index
cmpltScaledData[numerical_cols]=scaled_data[numerical_cols]    #overwrithing scaled value to the complete dataframe
cmpltScaledData

# Splitting DataSet

In [ ]:
X=cmpltScaledData.drop(columns="deposit")
Y=cmpltScaledData.deposit

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

# Model Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score


#hyper parameter tuning 

rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)



decision_tree=DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

k_neighbor=KNeighborsClassifier()
k_neighbor.fit(X_train, y_train)

svc_classifier=SVC()
svc_classifier.fit(X_train, y_train)

# Predicting the test data

In [ ]:
from sklearn.metrics import accuracy_score



accuracy=accuracy_score(y_test, grid_search.predict(X_test))

f1score= f1_score(y_test,grid_search.predict(X_test))
print(" Randomforest classification  \n \t f1 score = ",f1score)
print("\t accuracy = ",accuracy)
                                                                                
                                                                                
print(" DessionTree classification \n \t f1 score",f1_score(y_test,decision_tree.predict(X_test)))
print("\t accuracy = ",accuracy_score(y_test,decision_tree.predict(X_test)))

print(" Kneighbour classification \n \t f1 score ",f1_score(y_test,k_neighbor.predict(X_test)))
print("\t accuracy = ",accuracy_score(y_test,k_neighbor.predict(X_test)))

print(" svc classification \n \t f1 score",f1_score(y_test,svc_classifier.predict(X_test)))
print("\t accuracy = ",accuracy_score(y_test,svc_classifier.predict(X_test)))

# Result: Randomforest classifier has highest accuracy of 85% among four algorithms which i have used.